# Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import data_utils as utils
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean

# Variables

In [ ]:
target_well = 'LC-1C'
target_signal = 'wl'

# Load, Clean, & Add Data

In [ ]:
df = pd.read_csv('useful_well_data.csv')
df = utils.pivot_table_and_clean_dates(df)
df = utils.clean_values(df)

# Find nearest wells

In [ ]:
dft = df.stack(level=0).reset_index()
dft.columns = ['date', 'id', 'mean_temp', 'precip', 'wl']
dft['date'] = pd.to_datetime(dft['date'])
dft['id'] = dft['id'].astype(str)

In [ ]:
well_ids = dft.id.unique()
well_coor = pd.read_csv('oldData/Observations_coordinates.csv')
well_coor.columns = ['id', 'x', 'y']
well_coor = well_coor.set_index('id')
well_coor = well_coor.loc[well_ids]

In [ ]:
df_mod, well_names = utils.get_nearest_data(well_coor, target_well, df)
df_mod

#### Adding data to catch trends

In [ ]:
df_mod['dates'] = df_mod.index.values
df_mod['dates'] = pd.to_datetime(df_mod['dates'])
df_mod['trend'] = df_mod['dates'].dt.month
df_mod = df_mod.drop('dates', axis=1)


#### See data

In [ ]:
plt.plot(df_mod[target_well][target_signal], label=target_well+' (target well)')
plt.plot(df_mod[well_names[1]][target_signal], label=well_names[1])
plt.plot(df_mod[well_names[2]][target_signal], label=well_names[2])
plt.xlabel('Year')
plt.ylabel('Water Level')
plt.legend()
plt.show()

# Prepare Data for model
#### We want to predict 2 years, that's 24 timesteps we need to shift

In [ ]:
shift_steps = 24
df_targets = df_mod[target_well][target_signal].shift(-shift_steps)

#### Comparing the original and time-shifted data-frames
#### The last 5 entries of the original dataframe should be the same as the first 5 of the shifted one

In [ ]:
df_mod[target_well][target_signal].head(shift_steps+5)

In [ ]:
df_targets.head(5)

In [ ]:
df_targets.tail(25)

### Arrays and scaling

In [ ]:
x_data = df_mod.values[0:-shift_steps]
print(type(x_data))
print('Shape: ', x_data.shape)

In [ ]:
y_data = df_targets.values[:-shift_steps]
y_data = y_data.reshape(-1,1)
print(type(y_data))
print('Shape: ', y_data.shape)

In [ ]:
num_data = len(x_data)
num_data

In [ ]:
train_split = 0.8

In [ ]:
num_train = int(train_split*num_data)
num_train

In [ ]:
num_test = num_data - num_train
num_test

In [ ]:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

In [ ]:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
len(y_train) + len(y_test)
print(y_test.shape, y_train.shape)

In [ ]:
num_x_signals = x_data.shape[1]
num_y_signals = 1

##### Scale Data

In [ ]:
print('Min: ', np.min(x_train))
print('Max: ', np.max(x_train))

In [ ]:
x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [ ]:
print('Min: ', np.min(x_train_scaled))
print('Max: ', np.max(x_train_scaled))

In [ ]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled =  y_scaler.transform(y_test)

#### Data Generator

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)

            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]

        yield x_batch, y_batch

In [ ]:
batch_size, sequence_length = 8, 3

In [ ]:
generator = batch_generator(batch_size, sequence_length)

In [ ]:
x_batch, y_batch = next(generator)

In [ ]:
print(x_batch.shape)
print(y_batch.shape)

In [ ]:
batch, signal = 0, 0
seq = x_batch[batch, :, signal]
plt.plot(seq)

In [ ]:
seq = y_batch[batch, :, signal]
plt.plot(seq)

#### Validation Set

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

# Create the Network

In [ ]:
model = Sequential()
units = 64
model.add(GRU(units=units,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))
model.add(GRU(units=int(units/2),
              return_sequences=True,
              input_shape=(None, num_x_signals,)))
model.add(Dense(num_y_signals,
                activation='sigmoid'))

#### Loss Function

In [ ]:
warmup_steps = 12

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.

    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculate the Mean Squared Error and use it as loss.
    mse = mean(square(y_true_slice - y_pred_slice))

    return mse

### compile model

In [ ]:
optimizer = Adam(learning_rate=0.1)

In [ ]:
model.compile(loss=loss_mse_warmup, optimizer=optimizer)

In [ ]:
model.summary()

#### Callback Functions

In [ ]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [ ]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=10,
                                       verbose=1)

In [ ]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_reduce_lr]

### Train the Network

In [ ]:
validation_data[0].shape

In [ ]:
%%time
model.fit(x=generator,
          epochs=20,
          steps_per_epoch=100,
          validation_data=validation_data,
          callbacks=callbacks)

In [ ]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))

In [ ]:
print("loss (test-set):", result)


In [ ]:
def plot_comparison(start_idx, length, train=True):
    """
    Plot the predicted and true output-signals.

    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """

    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test

    # End-index for the sequences.
    end_idx = start_idx + length

    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]

    # Input-signals for the model.
    # x = np.expand_dims(x, axis=0)
    x = x.reshape(1, x.shape[0], x.shape[1])
    print(x.shape)

    # Use the model to predict the output-signals.
    #x = x.reshape(1, x.shape[0], x.shape[1])
    # x = n_x_train_scaled
    y_pred = model.predict(x)

    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])

    # For each output-signal.
    for signal in range(len(target_signal)):
        # Get the output-signal predicted by the model.
        signal_pred = y_pred_rescaled[:, signal]

        # Get the true output-signal from the data-set.
        signal_true = y_true[:, signal]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))

        # Plot and compare the two signals.
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')

        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)

        # Plot labels etc.
        plt.ylabel(target_signal[signal])
        plt.legend()
        plt.show()

In [ ]:
#x_train_scaled
n_x_train_scaled = x_train_scaled.reshape(1,220,6)
n_x_train_scaled.shape

In [ ]:
y_pred = model.predict(n_x_train_scaled)


In [ ]:
y_pred.shape


In [ ]:
plot_comparison(start_idx=169, length=50, train=True)
